#### Practicing AV [lesson](https://www.analyticsvidhya.com/blog/2020/06/feature-engineering-guide-data-science-hackathons/) on feature engineering

In [0]:
import numpy as np
import pandas as pd
from math import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
import matplotlib as plt
from sklearn.metrics import mean_squared_error, r2_score

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/Datasets/Black Friday sales'
train = pd.read_csv(path +'/train.csv')
test = pd.read_csv(path +'/test.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [4]:
train.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.500680e+05,550068.000000,550068.000000,550068.000000,376430.000000,166821.000000,550068.000000
mean,1.003029e+06,8.076707,0.409653,5.404270,9.842329,12.668243,9263.968713
std,1.727592e+03,6.522660,0.491770,3.936211,5.086590,4.125338,5023.065394
min,1.000001e+06,0.000000,0.000000,1.000000,2.000000,3.000000,12.000000
25%,1.001516e+06,2.000000,0.000000,1.000000,5.000000,9.000000,5823.000000
50%,1.003077e+06,7.000000,0.000000,5.000000,9.000000,14.000000,8047.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,15.000000,16.000000,12054.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


In [5]:
print(train['Gender'].unique())
print(train['City_Category'].unique())
print(train['Age'].unique())
print(train['Stay_In_Current_City_Years'].unique())
print(train['Product_ID'].unique())

['F' 'M']
['A' 'C' 'B']
['0-17' '55+' '26-35' '46-50' '51-55' '36-45' '18-25']
['2' '4+' '3' '1' '0']
['P00069042' 'P00248942' 'P00087842' ... 'P00370293' 'P00371644'
 'P00370853']


In [6]:
def missing_values_table(df):
       mis_val = df.isnull().sum()
       mis_val_percent = 100 * df.isnull().sum() / len(df)
       mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
       mis_val_table_ren_columns = mis_val_table.rename(
       columns = {0 : 'Missing Values', 1 : '% of Total Values'})
       mis_val_table_ren_columns = mis_val_table_ren_columns[
           mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
       '% of Total Values', ascending=False).round(1)
       print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"     
           "There are " + str(mis_val_table_ren_columns.shape[0]) +
             " columns that have missing values.")
       return mis_val_table_ren_columns
missing_values_table(train)

Your selected dataframe has 12 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
Product_Category_3,383247,69.7
Product_Category_2,173638,31.6


In [0]:
gender_dict = {'F':0, 'M':1}
age_dict = {'0-17':0, '18-25':1, '26-35':2, '36-45':3, '46-50':4, '51-55':5, '55+':6}
city_dict = {'A':0, 'B':1, 'C':2}
stay_dict = {'0':0, '1':1, '2':2, '3':3, '4+':4}
 
train["Gender"] = train["Gender"].apply(lambda x: gender_dict[x])
test["Gender"] = test["Gender"].apply(lambda x: gender_dict[x])
 
train["Age"] = train["Age"].apply(lambda x: age_dict[x])
test["Age"] = test["Age"].apply(lambda x: age_dict[x])
 
train["City_Category"] = train["City_Category"].apply(lambda x: city_dict[x])
test["City_Category"] = test["City_Category"].apply(lambda x: city_dict[x])
 
train["Stay_In_Current_City_Years"] = train["Stay_In_Current_City_Years"].apply(lambda x: stay_dict[x])
test["Stay_In_Current_City_Years"] = test["Stay_In_Current_City_Years"].apply(lambda x: stay_dict[x])

from sklearn.preprocessing import LabelEncoder
columns_list = ["User_ID", "Product_ID"]
for var in columns_list:
   lb = LabelEncoder()
   full_var_data = pd.concat((train[var],test[var]),axis=0).astype('str')
   temp = lb.fit_transform(np.array(full_var_data))
   list(lb.classes_)
   train[var] = lb.transform(np.array( train[var] ).astype('str'))
   test[var] = lb.transform(np.array( test[var] ).astype('str'))

In [0]:
train.fillna(-999, inplace=True)
test.fillna(-999, inplace=True)

In [9]:
## Check missing values
missing_values_table(train)

Your selected dataframe has 12 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [0]:
# SPlitting training data
def split_train_data(train):
  X = train.drop('Purchase',axis=1)
  y = train['Purchase']
  print(X.head())
  print(y.head())
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=3)
  return X,y,X_train, X_test, y_train, y_test


In [0]:
def model_DTR(X,X_train,y_train,X_test,y_test,test):
  dtr = DecisionTreeRegressor()
  dtr.fit(X_train,y_train)
  y_pred = dtr.predict(X_test)
  y_pred_dt = dtr.predict(test)
  mse = mean_squared_error(y_test, y_pred)
  print("RMSE Error:", np.sqrt(mse))
  r2 = r2_score(y_test, y_pred)
  print("R2 Score:", r2)
  feature_importance(dtr,X)
  return y_pred_dt
  

### Calculating feature importance

In [0]:
def feature_importance(dtr,X):
  importance = dtr.feature_importances_
  feature_importance = dict(zip(X.columns, importance ))
  keys = list(feature_importance.keys())
  values = list(feature_importance.values())
  total = sum(values)
  new = [value * 100. / total for value in values]
  new = np.round(new,2)
  importance_df = pd.DataFrame()
  importance_df['Features'] = keys
  importance_df['Importance (%)'] = new
  importance_df = importance_df.sort_values(['Importance (%)'],ascending=False).reset_index(drop=True)
  importance_df
  importance_df.style.set_properties(**{'font-size':'10pt'})
  print(importance_df)

### Fitting the first model

In [0]:
train_1 = train.copy()
test_1 = test.copy()

In [14]:
X,y,X_train, X_test, y_train, y_test = split_train_data(train_1)
y_pred_dt_1 = model_DTR(X, X_train,y_train, X_test, y_test,test_1)

   User_ID  Product_ID  ...  Product_Category_2  Product_Category_3
0        0         684  ...              -999.0              -999.0
1        0        2406  ...                 6.0                14.0
2        0         868  ...              -999.0              -999.0
3        0         844  ...                14.0              -999.0
4        1        2769  ...              -999.0              -999.0

[5 rows x 11 columns]
0     8370
1    15200
2     1422
3     1057
4     7969
Name: Purchase, dtype: int64
RMSE Error: 3753.84063401002
R2 Score: 0.44142935255600624
                      Features  Importance (%)
0           Product_Category_1           63.64
1                      User_ID           11.67
2                   Product_ID            7.85
3                   Occupation            4.82
4   Stay_In_Current_City_Years            3.02
5                          Age            2.83
6           Product_Category_2            1.57
7                City_Category            1.46
8  

### Modifying the dataset

In [0]:
train_2 = train_1.copy()
test_2 = test_1.copy()
train_2["User_ID_MeanPrice"] = train_2.groupby(['User_ID'])['Purchase'].transform('mean')
userID_mean_dict = train_2.groupby(['User_ID'])['Purchase'].mean().to_dict()
test_2['User_ID_MeanPrice'] = test_2['User_ID'].apply(lambda x:userID_mean_dict.get(x,0))

train_2["Product_ID_MeanPrice"] = train_2.groupby(['Product_ID'])['Purchase'].transform('mean')
productID_mean_dict = train_2.groupby(['Product_ID'])['Purchase'].mean().to_dict()
test_2['Product_ID_MeanPrice'] = test_2['Product_ID'].apply(lambda x:productID_mean_dict.get(x,0))





 ![alt text](https://www.w3resource.com/w3r_images/pandas-dataframe-to_dict.svg)

```df.to_dict()```

Output: {'c1': {'row1': 1, 'row2': 2}, 'c2': {'row1': 0.6, 'row2': 0.85}}

```dict.get(key, default = None)```

This method return a value for the given key. If key is not available, then returns default value None.

```df.get('c1',0)```

Output: {'row1': 1, 'row2': 2}

In [16]:
X,y,X_train, X_test, y_train, y_test = split_train_data(train_2)
y_pred_dt_2 = model_DTR(X,X_train,y_train, X_test, y_test,test_2)


   User_ID  Product_ID  ...  User_ID_MeanPrice  Product_ID_MeanPrice
0        0         684  ...        9545.514286          11870.863436
1        0        2406  ...        9545.514286          16304.030981
2        0         868  ...        9545.514286           1237.892157
3        0         844  ...        9545.514286           1455.140762
4        1        2769  ...       10525.610390           7692.763547

[5 rows x 13 columns]
0     8370
1    15200
2     1422
3     1057
4     7969
Name: Purchase, dtype: int64
RMSE Error: 3531.1975945571403
R2 Score: 0.5057229024042318
                      Features  Importance (%)
0         Product_ID_MeanPrice           75.48
1            User_ID_MeanPrice            6.77
2                      User_ID            4.22
3                   Product_ID            3.38
4                   Occupation            2.16
5           Product_Category_2            1.52
6                          Age            1.43
7   Stay_In_Current_City_Years            1

### Modification - Part 2

In [0]:
train_3 = train_2.copy()
test_3 = test_2.copy()

In [0]:
train_3["User_ID_MinPrice"] = train_3.groupby(['User_ID'])['Purchase'].transform('min')
userID_min_dict = train_3.groupby(['User_ID'])['Purchase'].min().to_dict()
test_3['User_ID_MinPrice'] = test_3['User_ID'].apply(lambda x:userID_min_dict.get(x,0))
 
train_3["User_ID_MaxPrice"] = train_3.groupby(['User_ID'])['Purchase'].transform('max')
userID_max_dict = train_3.groupby(['User_ID'])['Purchase'].max().to_dict()
test_3['User_ID_MaxPrice'] = test_3['User_ID'].apply(lambda x:userID_max_dict.get(x,0))
 
train_3["Product_ID_MinPrice"] = train_3.groupby(['Product_ID'])['Purchase'].transform('min')
productID_min_dict = train_3.groupby(['Product_ID'])['Purchase'].min().to_dict()
test_3['Product_ID_MinPrice'] = test_3['Product_ID'].apply(lambda x:productID_min_dict.get(x,0))

train_3["Product_ID_MaxPrice"] = train_3.groupby(['Product_ID'])['Purchase'].transform('max')
productID_max_dict = train_3.groupby(['Product_ID'])['Purchase'].max().to_dict()
test_3['Product_ID_MaxPrice'] = test_3['Product_ID'].apply(lambda x:productID_max_dict.get(x,0))

In [19]:
X,y,X_train, X_test, y_train, y_test = split_train_data(train_3)
y_pred_dt_3 = model_DTR(X,X_train,y_train, X_test, y_test,test_3)

   User_ID  Product_ID  ...  Product_ID_MinPrice  Product_ID_MaxPrice
0        0         684  ...                 2648                13716
1        0        2406  ...                 3880                19701
2        0         868  ...                  343                 1776
3        0         844  ...                  365                 1778
4        1        2769  ...                 3920                10073

[5 rows x 17 columns]
0     8370
1    15200
2     1422
3     1057
4     7969
Name: Purchase, dtype: int64
RMSE Error: 3489.392228189329
R2 Score: 0.5173569833684961
                      Features  Importance (%)
0         Product_ID_MeanPrice           74.30
1            User_ID_MeanPrice            5.07
2             User_ID_MaxPrice            2.92
3                      User_ID            2.55
4             User_ID_MinPrice            2.40
5                   Product_ID            2.20
6          Product_ID_MinPrice            2.04
7          Product_ID_MaxPrice        

In [0]:
train_4 = train_3.copy()
test_4 = test_3.copy()

In [0]:
train_4["Product_Cat1_MaxPrice"] = train_4.groupby(['Product_Category_1'])['Purchase'].transform('max')
pc1_max_dict = train_4.groupby(['Product_Category_1'])['Purchase'].max().to_dict()
test_4['Product_Cat1_MaxPrice'] = test_4['Product_Category_1'].apply(lambda x:pc1_max_dict.get(x,0))
 
train_4["Product_Cat1_MeanPrice"] = train_4.groupby(['Product_Category_1'])['Purchase'].transform('mean')
pc1_mean_dict = train_4.groupby(['Product_Category_1'])['Purchase'].mean().to_dict()
test_4['Product_Cat1_MeanPrice'] = test_4['Product_Category_1'].apply(lambda x:pc1_mean_dict.get(x,0))

train_4["Age_Count"] = train_4.groupby(['Age'])['Age'].transform('count')
age_count_dict = train_4.groupby(['Age']).size().to_dict()
test_4['Age_Count'] = test_4['Age'].apply(lambda x:age_count_dict.get(x,0))
 
train_4["Occupation_Count"] = train_4.groupby(['Occupation'])['Occupation'].transform('count')
occupation_count_dict = train_4.groupby(['Occupation']).size().to_dict()
test_4['Occupation_Count'] = test_4['Occupation'].apply(lambda x:occupation_count_dict.get(x,0))

train_4["Product_Category_1_Count"] = train_4.groupby(['Product_Category_1'])['Product_Category_1'].transform('count')
pc1_count_dict = train_4.groupby(['Product_Category_1']).size().to_dict()
test_4['Product_Category_1_Count'] = test_4['Product_Category_1'].apply(lambda x:pc1_count_dict.get(x,0))
 
train_4["Product_Category_2_Count"] = train_4.groupby(['Product_Category_2'])['Product_Category_2'].transform('count')
pc2_count_dict = train_4.groupby(['Product_Category_2']).size().to_dict()
test_4['Product_Category_2_Count'] = test_4['Product_Category_2'].apply(lambda x:pc2_count_dict.get(x,0))
 
train_4["Product_Category_3_Count"] = train_4.groupby(['Product_Category_3'])['Product_Category_3'].transform('count')
pc3_count_dict = train_4.groupby(['Product_Category_3']).size().to_dict()
test_4['Product_Category_3_Count'] = test_4['Product_Category_3'].apply(lambda x:pc3_count_dict.get(x,0))
 
train_4["User_ID_Count"] = train_4.groupby(['User_ID'])['User_ID'].transform('count')
userID_count_dict = train_4.groupby(['User_ID']).size().to_dict()
test_4['User_ID_Count'] = test_4['User_ID'].apply(lambda x:userID_count_dict.get(x,0))
 
train_4["Product_ID_Count"] = train_4.groupby(['Product_ID'])['Product_ID'].transform('count')
productID_count_dict = train_4.groupby(['Product_ID']).size().to_dict()
test_4['Product_ID_Count'] = test_4['Product_ID'].apply(lambda x:productID_count_dict.get(x,0))

In [22]:
X,y,X_train, X_test, y_train, y_test = split_train_data(train_4)
y_pred_dt_4 = model_DTR(X,X_train,y_train, X_test, y_test,test_4)

   User_ID  Product_ID  ...  User_ID_Count  Product_ID_Count
0        0         684  ...             35               227
1        0        2406  ...             35               581
2        0         868  ...             35               102
3        0         844  ...             35               341
4        1        2769  ...             77               203

[5 rows x 26 columns]
0     8370
1    15200
2     1422
3     1057
4     7969
Name: Purchase, dtype: int64
RMSE Error: 3499.5433357915986
R2 Score: 0.5145447528583189
                      Features  Importance (%)
0         Product_ID_MeanPrice           73.79
1            User_ID_MeanPrice            4.49
2             User_ID_MaxPrice            2.28
3                User_ID_Count            2.04
4                      User_ID            1.92
5             User_ID_MinPrice            1.77
6                   Product_ID            1.71
7             Product_ID_Count            1.66
8          Product_ID_MinPrice            1.

### Creating submission CSV file

In [0]:
def create_submission_file(y_pred_dt,path=path):
  submission = pd.DataFrame()
  submission['User_ID'] = test['User_ID']
  submission['Product_ID'] = test['Product_ID']
  submission['Purchase'] = y_pred_dt
  submission.to_csv(path +'/test_submission.csv',
                  index=False)

In [0]:
create_submission_file(y_pred_dt_4)